<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [10]:
%reload_ext sql

And then we import our movie database

In [27]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [28]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [11]:
%%sql
SELECT title
FROM   movies

 * sqlite:///movies.sqlite
Done.


title
House of Gucci
Madres paralelas
Death of the Nile
Hytti nro 6


+ Show the performance dates for one of the movies.

In [14]:
%%sql
SELECT      start_date
FROM        performances
WHERE       imdb IN (
        SELECT imdb
        FROM   movies
        LIMIT 1
)


 * sqlite:///movies.sqlite
Done.


start_date
2022-02-11
2022-02-13


+ Show all data concerning performances at a given theatere
  on a given date.

In [17]:
%%sql
SELECT  *
FROM    performances
WHERE   name = 'Södran' AND start_date = '2022-02-11'

 * sqlite:///movies.sqlite
Done.


p_id,start_time,start_date,imdb,name
728a759eb2f1b18e58a7d12ab70320f9,13:00,2022-02-11,034256,Södran
60b1fc3b83a00530c7eec189aa013f54,15:00,2022-02-11,937548,Södran


+ List all customers

In [25]:
%%sql
SELECT *
FROM   customers

 * sqlite:///movies.sqlite
Done.


user_name,full_name,password
4NGIEinCAP5,Angelica Larsson,an7061la-s
cerealkiller1337,Märta Holmquist,ma7061ho-s


+ List all tickets

In [57]:
%%sql
SELECT *
FROM   tickets

 * sqlite:///movies.sqlite
Done.


ticket_id,p_id,user_name
8803c67b342be960cd7f18afe010e031,c972ad3788214c530e95a88680a8d62b,4NGIEinCAP5
7a7a4bd5108f00614fb00fb3020aa2e9,c972ad3788214c530e95a88680a8d62b,4NGIEinCAP5
744cf7f5d1e1ebf173ff0b2238e37431,c972ad3788214c530e95a88680a8d62b,4NGIEinCAP5


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [56]:
%%sql
INSERT
INTO    tickets (p_id, user_name)
VALUES  ('c972ad3788214c530e95a88680a8d62b', '4NGIEinCAP5');

 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [66]:
%%sql
SELECT ticket_id
FROM   tickets
WHERE  rowid = last_insert_rowid();

 * sqlite:///movies.sqlite
Done.


ticket_id


+ Try to insert two movie theaters with the same name (this
  should fail).

In [59]:
%%sql
INSERT 
INTO theatres (name, capacity)
VALUES ('SF', 100),
       ('SF', 100);

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theatres.name
[SQL: INSERT INTO theatres (name, capacity)
VALUES ('SF', 100),
       ('SF', 100);]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [63]:
%%sql
INSERT 
INTO performances (start_time, start_date, imdb, name)
VALUES ('16:00', '2022-02-11', '165721', 'Biograf');

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO performances (start_time, start_date, imdb, name)
VALUES ('16:00', '2022-02-11', '165721', 'Biograf');]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [64]:
%%sql
INSERT
INTO tickets (p_id, user_name)
VALUES ('c972ad3788214c530e95a88680a8d62b', 'y4z4n');

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets (p_id, user_name)
VALUES ('c972ad3788214c530e95a88680a8d62b', 'y4z4n');]
(Background on this error at: https://sqlalche.me/e/14/gkpj)